In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import os
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time

In [ ]:
ts = TimeSeries(key=api_key, output_format='pandas')
df, meta_data = ts.get_intraday(symbol='MSFT', interval = '5min', outputsize =␣
↪'full')
print(df)


In [ ]:
# For the default date string index behavior
ts = TimeSeries(key=api_key,output_format='pandas', indexing_type='date')
ts
print(ts)

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
ts = TimeSeries(key=api_key, output_format='pandas')
data, meta_data = ts.get_intraday(symbol='MSFT',interval='5min',␣
↪outputsize='full')
print(data)


In [ ]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
ts = TimeSeries(key='IKGHGW5MQOYG9QOL', output_format='pandas')
df, meta_data = ts.get_intraday(symbol='MSFT',interval='5min',␣
↪outputsize='full')
df['4. close'].plot()
plt.title('Intraday Times Series for the MSFT stock (5 min)')
plt.show()

In [ ]:
df, meta_data = ts.get_intraday(symbol='MSFT', interval = '1min', outputsize =␣
↪'full')
df.to_csv("Dataset\Out.csv")


In [ ]:
df = pd.read_csv("Dataset\Out.csv")
df.head()

In [ ]:
data=df.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(df)),columns=['date','4. close'])
for i in range(0,len(data)):
new_dataset["date"][i]=df['date'][i]
new_dataset["4. close"][i]=df["4. close"][i]

In [ ]:
new_dataset.index=new_dataset.date
new_dataset.drop("date",axis=1,inplace=True)
final_dataset=new_dataset.values
train_data=final_dataset[0:987,:]
valid_data=final_dataset[987:,:]
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(final_dataset)
x_train_data,y_train_data=[],[]
for i in range(60,len(train_data)):
x_train_data.append(scaled_data[i-60:i,0])
y_train_data.append(scaled_data[i,0])
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.
↪shape[1],1))

In [ ]:
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.
↪shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=10,batch_size=1,verbose=2)
inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

In [ ]:
X_test=[]
for i in range(60,inputs_data.shape[0]):
 X_test.append(inputs_data[i-60:i,0])
X_test=np.array(X_test)


In [ ]:
print(X_test.size)

In [ ]:
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=lstm_model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)
predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

In [ ]:
lstm_model.save("saved_lstm_model.h5")

In [ ]:
train_data=new_dataset[:987]
valid_data=new_dataset[987:]
valid_data['Predictions']=predicted_closing_price
plt.plot(train_data["4. close"])
plt.plot(valid_data[['4. close',"Predictions"]])

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math


In [ ]:
lstm_model = tf.keras.models.load_model("saved_lstm_model.h5")

In [ ]:
inputs_data = new_dataset[len(new_dataset)-len(valid_data)-60:].values
inputs_data = inputs_data.reshape(-1,1)
inputs_data = scaler.transform(inputs_data)
X_test = []
for i in range(60, inputs_data.shape[0]):
X_test.append(inputs_data[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted_closing_price = lstm_model.predict(X_test)
predicted_closing_price = scaler.inverse_transform(predicted_closing_price)

In [ ]:
true_closing_price = valid_data['4. close'].values
mse = mean_squared_error(true_closing_price, predicted_closing_price)
mae = mean_absolute_error(true_closing_price, predicted_closing_price)
rmse = math.sqrt(mse)

In [ ]:
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
plt.plot(valid_data.index, true_closing_price, label='Actual Closing Price')
plt.plot(valid_data.index, predicted_closing_price, label='Predicted Closing␣
↪Price')
plt.title('Actual vs. Predicted Closing Prices')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend

In [ ]:
import matplotlib.pyplot as plt
# Data
models = ['Linear regression', 'MLP Regressor', 'RNN', 'LSTM', 'Custom LSTM']
rmse = [417.757, 391.752, 257.52, 78.81, 55.33]
# Create bar chart
plt.figure(figsize=(10, 6))
plt.bar(models, rmse, color=['blue', 'orange', 'green', 'red', 'purple'])
plt.xlabel('Models')
plt.ylabel('RMSE')
plt.title('Comparison of RMSE among different models')
plt.xticks(rotation=45)
plt.tight_layout()
# Show plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
# Data
models = ['Linear regression', 'MLP Regressor', 'RNN', 'LSTM', 'Custom LSTM']
rmse = [417.757, 391.752, 257.52, 78.81, 55.33]
# Colors
colors = ['#1f77b4', '#1f77b4', '#1f77b4', '#1f77b4', '#d62728'] # Dark blue␣
↪for all bars, red for Custom LSTM
# Create bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(models, rmse, color=colors, width=0.5) # Adjust the width for␣
↪thinner bars
# Highlighting the "Custom LSTM" bar with a different color
bars[-1].set_color('#d62728')
# Add data labels
for bar, value in zip(bars, rmse):
plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5, f'{value:.
↪2f}', ha='center', va='bottom')
# Customize chart appearance
plt.xlabel('Models', fontsize=12)
plt.ylabel('RMSE', fontsize=12)
plt.title('Comparison of RMSE among different models', fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# Show plot
plt.tight_layout()
plt.show()
